In [1]:
import os
import pandas as pd
import numpy as np
from pathlib import Path
import time
from utils.basefuncs import preProcessDataFrame

In [2]:
path = r"D:\MSc\MSBD 5014\FindingPointsOfInterestFromGPS"
compiled_path = Path(path,"Configs.xlsx").resolve()

In [3]:
preprocess = preProcessDataFrame(compiled_path,sheetname='preProcessDataFrame')
df = preprocess.processDF()

In [4]:
df

,rmac,uuid,latitude,longitude,Hour,Minute,Second,Day,Month,Year,ts,ts,lost
0,rry0763mvkv84bhkl4ykk79k6xkchs-SM-A7050,211205,22.387185,113.974245,17,14,17,31,12,2022,1672506857,2022-12-31 17:14:17,0.0
1,8advjcqra9c4us8tvyupjxchwina35-SM-A127F,113805,22.344644,114.190896,21,22,47,31,12,2022,1672521767,2022-12-31 21:22:47,0.0
2,skc0fc3cp5pulh6z0yllw4no2dk0ag-SM-A125F,111287,22.323926,114.257628,7,37,50,1,1,2023,1672558670,2023-01-01 07:37:50,0.0
3,hbzdcb6zbzht4r6dkk5zbpgfpuo3xd-SM-A217F,112250,22.351677,114.131214,8,22,48,1,1,2023,1672561368,2023-01-01 08:22:48,0.0
4,vh2rxc9exuc923wi39ahicn14dzcls-moto e40,112727,22.296712,114.240309,9,47,10,1,1,2023,1672566430,2023-01-01 09:47:10,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2108234,xcJQkdEMilT6BdZLVotETym343eYTE-IOS,113324,22.361319,114.128374,3,23,26,23,8,2023,1692761006,2023-08-23 03:23:26,0.0
2108235,y4saPsrWmBVlPG16YP3RigihCSSHcj-IOS,112459,22.245235,114.148077,3,29,42,23,8,2023,1692761382,2023-08-23 03:29:42,0.0
2108236,GQYWWGBkay0oVY0eEtl8nk3EYZNpWm-IOS,112459,22.244295,114.148802,3,33,9,23,8,2023,1692761589,2023-08-23 03:33:09,0.0
2108237,GQYWWGBkay0oVY0eEtl8nk3EYZNpWm-IOS,112459,22.245249,114.148248,3,46,20,23,8,2023,1692762380,2023-08-23 03:46:20,0.0
